#Set-Up

In [4]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

# Procedure Pricing Optimization Model

This model uses **Mixed-Integer Linear Programming (MILP)** to determine fair and cost-effective reimbursement rates for medical procedures.
Each provider is assigned to one of several pricing tiers (e.g., low, mid, high) based on compliance limits, cost floors, and fairness rules.

The goal is to minimize total spending across all providers and procedures while:

* keeping prices within allowed regulatory bands,   
* respecting provider-specific minimum reimbursement levels, and  
* ensuring that price differences between providers stay within fairness caps.  

In short, the model finds the **lowest-cost combination of prices** that still meets all business, fairness, and compliance requirements.


In [7]:
# --- 1) Data ---
Providers  = ['A_Ochsner', 'B_BR_General', 'C_Lafayette_General']
Procedures = ['MRI', 'CT']
Regions    = ['BatonRouge']
Tiers      = ['T1_low', 'T2_mid', 'T3_high']

prov_region = {
    'A_Ochsner': 'BatonRouge',
    'B_BR_General': 'BatonRouge',
    'C_Lafayette_General': 'BatonRouge',
}

b = {('MRI','BatonRouge'): 900.0, ('CT','BatonRouge'): 600.0}
lam_lo = {('MRI','BatonRouge'): 0.85, ('CT','BatonRouge'): 0.85}
lam_hi = {('MRI','BatonRouge'): 1.15, ('CT','BatonRouge'): 1.15}

s = {}
for r in Regions:
    s[('MRI', r, 'T1_low')] = 800.0
    s[('MRI', r, 'T2_mid')] = 900.0
    s[('MRI', r, 'T3_high')] = 1000.0
    s[('CT',  r, 'T1_low')] = 540.0
    s[('CT',  r, 'T2_mid')] = 600.0
    s[('CT',  r, 'T3_high')] = 660.0

v = {
    ('A_Ochsner','MRI'): 600, ('A_Ochsner','CT'): 400,
    ('B_BR_General','MRI'): 400, ('B_BR_General','CT'): 300,
    ('C_Lafayette_General','MRI'): 300, ('C_Lafayette_General','CT'): 200,
}

c_floor = {
    ('A_Ochsner','MRI'): 800.0, ('A_Ochsner','CT'): 540.0,
    ('B_BR_General','MRI'): 780.0, ('B_BR_General','CT'): 520.0,
    ('C_Lafayette_General','MRI'): 820.0, ('C_Lafayette_General','CT'): 560.0,
}

Delta = {('MRI','BatonRouge'): 150.0, ('CT','BatonRouge'): 150.0}

S0 = sum(v[(i,p)] * b[(p, prov_region[i])] for i in Providers for p in Procedures)
tau = -0.03  # target >= 3% savings

# --- 2) Model ---
m = ConcreteModel()
m.I = Set(initialize=Providers)
m.P = Set(initialize=Procedures)
m.R = Set(initialize=Regions)
m.T = Set(initialize=Tiers)

def region_of(i): return prov_region[i]

# -----------------------------
# DECISION VARIABLES
# -----------------------------
# a[i,p,t] ∈ {0,1}: choose exactly one tier t for provider i, procedure p
m.a = Var(m.I, m.P, m.T, domain=Binary)

# x[i,p] ≥ 0: resulting reimbursement price for provider i, procedure p
m.x = Var(m.I, m.P, domain=NonNegativeReals)

# x_min[p,r], x_max[p,r] ≥ 0: min/max price in (procedure p, region r) for fairness
m.x_min = Var(m.P, m.R, domain=NonNegativeReals)
m.x_max = Var(m.P, m.R, domain=NonNegativeReals)

# -----------------------------
# OBJECTIVE FUNCTION
# -----------------------------
# Minimize total expected spend:  minimize Σ_{i,p} v[i,p] * x[i,p]
def total_spend(m):
    return sum(v[(i,p)] * m.x[i,p] for i in m.I for p in m.P)
m.OBJ = Objective(rule=total_spend, sense=minimize)

# -----------------------------
# CONSTRAINTS
# -----------------------------

# (1) Exactly one tier per provider–procedure:
#     Σ_t a[i,p,t] = 1  ∀i,p
def one_tier(m, i, p):
    return sum(m.a[i,p,t] for t in m.T) == 1
m.OneTier = Constraint(m.I, m.P, rule=one_tier)

# (2) Price equals chosen tier price:
#     x[i,p] = Σ_t s[p, r(i), t] * a[i,p,t]  ∀i,p
def price_link(m, i, p):
    r = region_of(i)
    return m.x[i,p] == sum(s[(p,r,t)] * m.a[i,p,t] for t in m.T)
m.PriceLink = Constraint(m.I, m.P, rule=price_link)

# (3) Compliance band around benchmark:
#     lam_lo[p,r]*b[p,r] ≤ x[i,p] ≤ lam_hi[p,r]*b[p,r]  ∀i,p
def band_lo_con(m, i, p):
    r = region_of(i)
    return m.x[i,p] >= lam_lo[(p,r)] * b[(p,r)]
def band_hi_con(m, i, p):
    r = region_of(i)
    return m.x[i,p] <= lam_hi[(p,r)] * b[(p,r)]
m.BandLo = Constraint(m.I, m.P, rule=band_lo_con)
m.BandHi = Constraint(m.I, m.P, rule=band_hi_con)

# (4) Provider-specific reimbursement floors:
#     x[i,p] ≥ c_floor[i,p]  ∀i,p
def floor_con(m, i, p):
    return m.x[i,p] >= c_floor[(i,p)]
m.Floor = Constraint(m.I, m.P, rule=floor_con)

# (5) Budget / savings target:
#     Σ_{i,p} v[i,p] * x[i,p] ≤ (1+tau)*S0
def budget_con(m):
    return sum(v[(i,p)] * m.x[i,p] for i in m.I for p in m.P) <= (1.0 + tau) * S0
m.Budget = Constraint(rule=budget_con)

# (6) Fairness spread per (p,r):
#     x_max[p,r] - x_min[p,r] ≤ Delta[p,r]
def fairness_spread(m, p, r):
    return m.x_max[p,r] - m.x_min[p,r] <= Delta[(p,r)]
m.FairnessSpread = Constraint(m.P, m.R, rule=fairness_spread)

# (7) Tie each provider price to the region min/max (defines min/max):
#     x[i,p] ≥ x_min[p,r(i)]  and  x[i,p] ≤ x_max[p,r(i)]   ∀i,p
def fairness_min_lb(m, i, p):
    r = region_of(i)
    return m.x[i,p] >= m.x_min[p,r]
def fairness_max_ub(m, i, p):
    r = region_of(i)
    return m.x[i,p] <= m.x_max[p,r]
m.FairMinLB = Constraint(m.I, m.P, rule=fairness_min_lb)
m.FairMaxUB = Constraint(m.I, m.P, rule=fairness_max_ub)

# --- 3) Solve (CBC) ---
opt = SolverFactory('cbc')
opt.options['seconds'] = 60        # stop after 60 seconds
opt.options['ratioGap'] = 0.001    # 0.1% MIP gap (integer optimality)
results = opt.solve(m, tee=False)  # set tee=True to see CBC log

# --- 4) Results ---
total_cost = value(m.OBJ)
print(f"Baseline spend (S0): ${S0:,.0f}")
print(f"Optimized spend:      ${total_cost:,.0f}")
print(f"Savings:              ${S0-total_cost:,.0f} ({100*(S0-total_cost)/S0:.1f}%)\n")

def chosen_tier(i,p):
    for t in m.T:
        if value(m.a[i,p,t]) > 0.5:
            return t
    return None

print("Final prices and tiers:")
for i in m.I:
    for p in m.P:
        r = region_of(i)
        tier = chosen_tier(i,p)
        price = value(m.x[i,p])
        print(f"  Provider={i:22s} Proc={p:3s} Region={r:10s}  Tier={tier:7s}  Price=${price:,.2f}")

print("\nFairness spreads (max - min per procedure-region):")
for p in m.P:
    for r in m.R:
        spread = value(m.x_max[p,r] - m.x_min[p,r])
        print(f"  {p} @ {r}: spread = ${spread:,.2f}  (cap {Delta[(p,r)]})")

Baseline spend (S0): $1,710,000
Optimized spend:      $1,568,000
Savings:              $142,000 (8.3%)

Final prices and tiers:
  Provider=A_Ochsner              Proc=MRI Region=BatonRouge  Tier=T1_low   Price=$800.00
  Provider=A_Ochsner              Proc=CT  Region=BatonRouge  Tier=T1_low   Price=$540.00
  Provider=B_BR_General           Proc=MRI Region=BatonRouge  Tier=T1_low   Price=$800.00
  Provider=B_BR_General           Proc=CT  Region=BatonRouge  Tier=T1_low   Price=$540.00
  Provider=C_Lafayette_General    Proc=MRI Region=BatonRouge  Tier=T2_mid   Price=$900.00
  Provider=C_Lafayette_General    Proc=CT  Region=BatonRouge  Tier=T2_mid   Price=$600.00

Fairness spreads (max - min per procedure-region):
  MRI @ BatonRouge: spread = $100.00  (cap 150.0)
  CT @ BatonRouge: spread = $60.00  (cap 150.0)


### Result Interpretation

The model reduced total spending from **\$1.71m to \$1.57m**, saving about **\$142K (8.3%)** while staying within all fairness and compliance rules.

* Both **Ochsner** and **Baton Rouge General** were assigned the lowest price tier (Tier 1) for MRI and CT since their costs met all requirements.
* **Lafayette General** had higher minimum reimbursement floors, so it was assigned to Tier 2 for both MRI and CT.
* All prices stayed within the legal range (85–115 % of the benchmark).
* The price differences between hospitals (spreads) were **\$100 for MRI** and **\$60 for CT**, both under the allowed cap of \$150.

In short, the solver chose the cheapest valid prices for each provider without violating any business or regulatory limits.
